In [1]:
import pandas as pd
import os
import joblib
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import logging

DATA_PATH = '../data/industrial_load_hourly.parquet'
MODEL_DIR = '../models/'
MODEL_PATH = os.path.join(MODEL_DIR, 'prophet_model.pkl')
TEST_YEAR = 2014

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

def load_data(path):
    """Load processed parquet data and rename columns for Prophet."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Data file '{path}' not found. Run preprocessing first.")
    df = pd.read_parquet(path)
    return df.reset_index().rename(columns={'timestamp': 'ds', 'load': 'y'})

def train_prophet(train_df):
    """Initialize and train Prophet model."""
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='multiplicative'
    )
    model.fit(train_df)
    return model

def forecast_and_evaluate(model, test_df):
    """Generate forecast for test period and compute evaluation metrics."""
    future = model.make_future_dataframe(periods=len(test_df), freq='h', include_history=False)
    forecast = model.predict(future)
    test_df = test_df.set_index('ds')
    test_df['yhat'] = forecast.set_index('ds')['yhat']
    rmse = np.sqrt(mean_squared_error(test_df['y'], test_df['yhat']))
    return test_df.reset_index(), rmse

def main():
    logging.info(" Loading data...")
    df = load_data(DATA_PATH)

    train_df = df[df['ds'].dt.year < TEST_YEAR]
    test_df = df[df['ds'].dt.year == TEST_YEAR]
    logging.info(f"Data split → Train: {train_df.shape}, Test: {test_df.shape}")

    logging.info(" Training Prophet model...")
    model = train_prophet(train_df)
    logging.info(" Prophet training complete.")

    logging.info(" Forecasting and evaluating model...")
    test_results, rmse = forecast_and_evaluate(model, test_df)
    logging.info(f" RMSE on test set: {rmse:.2f}")

    os.makedirs(MODEL_DIR, exist_ok=True)
    joblib.dump(model, MODEL_PATH)
    logging.info(f" Model saved to {MODEL_PATH}")

    logging.info("First 5 predictions:")
    logging.info(f"\n{test_results[['ds', 'y', 'yhat']].head()}")

if __name__ == "__main__":
    main()


2025-10-29 23:36:44,040 [INFO] 🚀 Loading data...
2025-10-29 23:36:44,156 [INFO] Data split → Train: (26304, 2), Test: (8760, 2)
2025-10-29 23:36:44,158 [INFO] 📈 Training Prophet model...
2025-10-29 23:36:47,656 [INFO] Chain [1] start processing
2025-10-29 23:37:21,043 [INFO] Chain [1] done processing
2025-10-29 23:37:21,224 [INFO] ✅ Prophet training complete.
2025-10-29 23:37:21,227 [INFO] 🔮 Forecasting and evaluating model...
2025-10-29 23:37:26,306 [INFO] 📊 RMSE on test set: 27.61
2025-10-29 23:37:26,572 [INFO] 💾 Model saved to ../models/prophet_model.pkl
2025-10-29 23:37:26,575 [INFO] First 5 predictions:
2025-10-29 23:37:26,584 [INFO] 
                   ds          y       yhat
0 2014-01-01 00:00:00  10.152284  25.744329
1 2014-01-01 01:00:00  11.421320  25.499928
2 2014-01-01 02:00:00  11.421320  26.059994
3 2014-01-01 03:00:00  11.421320  26.536283
4 2014-01-01 04:00:00  10.152284  26.119496


In [3]:
import pandas as pd
import numpy as np
import joblib
from prophet import Prophet

print("="*60)
print(" Phase 2: Prophet Baseline and Residual Calculation")
print("="*60)

data_path = '../data/industrial_load_hourly.parquet'
split_date = pd.to_datetime('2014-01-01 00:00:00')
TIME_STEPS = 168 # Lookback window for LSTM

try:
    df_full = (
        pd.read_parquet(data_path)
        .reset_index()
        .rename(columns={'timestamp': 'ds', 'load': 'y'})
    )
except FileNotFoundError:
    print(f" Using mock data for Prophet (Prophet import is still required).")
    date_range = pd.date_range(start='2011-01-01', end='2014-06-01', freq='H')
    df_full = pd.DataFrame({'ds': date_range, 'y': np.sin(np.arange(len(date_range))/100) * 1000 + 10000 + np.random.randn(len(date_range)) * 50})
    df_full['y'] = df_full['y'].apply(lambda x: max(0, x))


df_train = df_full[df_full['ds'] < split_date].copy()
df_test = df_full[df_full['ds'] >= split_date].copy()

m_prophet = Prophet(
    yearly_seasonality=True, 
    weekly_seasonality=True, 
    daily_seasonality=True, 
    seasonality_mode='multiplicative'
)
m_prophet.fit(df_train)
print(" Prophet model fitted on training data.")

future = df_full[['ds']].copy()
forecast_full = m_prophet.predict(future)
forecast_full = forecast_full[['ds', 'yhat']].rename(columns={'yhat': 'y_prophet'})

df_hybrid = df_full.merge(forecast_full, on='ds', how='left')
df_hybrid['residual'] = df_hybrid['y'] - df_hybrid['y_prophet']
print(" Residuals (y - y_prophet) calculated.")

start_index = df_hybrid[df_hybrid['ds'] == split_date].index[0] - TIME_STEPS
df_lstm_input = df_hybrid.iloc[start_index:].copy().reset_index(drop=True)

hybrid_components = {
    'prophet_model': m_prophet,
    'residual_df': df_lstm_input,
}
joblib.dump(hybrid_components, '../models/prophet_components.pkl')
print(f" Saved Prophet model and residual data to '../models/prophet_components.pkl'")

print("="*60)
print(" Phase 2 Complete: Residuals ready for LSTM training.")
print("="*60)


📊 Phase 2: Prophet Baseline and Residual Calculation


15:27:23 - cmdstanpy - INFO - Chain [1] start processing
15:28:47 - cmdstanpy - INFO - Chain [1] done processing


✅ Prophet model fitted on training data.
✅ Residuals (y - y_prophet) calculated.
📁 Saved Prophet model and residual data to '../models/prophet_components.pkl'
✅ Phase 2 Complete: Residuals ready for LSTM training.
